Whisper Models
- Tiny
- Small
- Medium
- Large
- Large-v2

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul  2 09:18:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   46C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -U datasets huggingface_hub fsspec evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.0
    Uninstalling huggingface-hub-0.33.0:
      Successfully uninstalled huggingface-hub-0.33.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency 

In [3]:
from datasets import Audio, load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load

In [4]:
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium").to("cuda")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="dutch", task="transcribe")

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [5]:
data = load_dataset("bchiusano/AsymmetriesCHILDES", 'correctPatternsSix')
data_sampled = data['train'].cast_column("audio", Audio(sampling_rate=16_000))
print(data)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/20.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/528 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'file_name', 'transcript'],
        num_rows: 528
    })
})


In [6]:
data_sampled[0]

{'audio': {'path': 'audio_c20_740_3620.mp3',
  'array': array([-0.00237633,  0.02671747,  0.03978223, ..., -0.00588393,
         -0.00784106, -0.01021122]),
  'sampling_rate': 16000},
 'file_name': 'c20.mp3',
 'transcript': 'een meisje die gaat met de gieter '}

In [8]:
def prepare_dataset(batch):
  if batch['transcript'] != 'hij wilde er niet meer vanaf ':
    audio = batch["audio"]
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcript"])

    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return batch

In [9]:
data_sampled = data_sampled.map(prepare_dataset)

Map:   0%|          | 0/528 [00:00<?, ? examples/s]

In [10]:
print(data_sampled)

Dataset({
    features: ['audio', 'file_name', 'transcript', 'input_features', 'labels', 'input_length'],
    num_rows: 527
})


In [11]:
MIN_DURATION_IN_SECONDS = 3.0

def is_audio_length_in_range(input_length):
    return input_length > MIN_DURATION_IN_SECONDS

In [12]:
data_sampled = data_sampled.filter(is_audio_length_in_range, input_columns=["input_length"])


Filter:   0%|          | 0/527 [00:00<?, ? examples/s]

In [13]:
data_sampled

Dataset({
    features: ['audio', 'file_name', 'transcript', 'input_features', 'labels', 'input_length'],
    num_rows: 344
})

In [14]:
def map_to_pred(batch):
    audio = batch["audio"]

    #input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
    input_features = inputs.input_features
    attention_mask = inputs.get("attention_mask")
    batch["reference"] = processor.tokenizer._normalize(batch['transcript'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"), attention_mask=attention_mask, forced_decoder_ids=forced_decoder_ids)[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

In [ ]:
result = data_sampled.map(map_to_pred)

In [17]:
import pandas as pd

output_path='asr_results.xlsx'
records = []

for i in range(len(result)):
    record = {
        'Index': i,
        'File Name': data_sampled[i]['file_name'],
        'Original Transcript': data_sampled[i]['transcript'],
        'Reference': result[i]['reference'],
        'Prediction': result[i]['prediction']
    }
    records.append(record)

    print(i)
    print(f"File Name: {data_sampled[i]['file_name']}")
    print(f"Original Transcript: {data_sampled[i]['transcript']}")
    print(f"Reference: {result[i]['reference']}")
    print(f"Prediction: {result[i]['prediction']}")
    print("")

df = pd.DataFrame(records)
df.to_excel(output_path, index=False)
print(f"Results saved to {output_path}")

0
File Name: c20.mp3
Original Transcript: die doet die gaat de kraan opendoen 
Reference: die doet die gaat de kraan opendoen
Prediction: die gaat klaar open doen

1
File Name: c20.mp3
Original Transcript: en dan gaat hij dan gaat de dokter en de meisje bloemetjes zaaien bloemetjes xxx 
Reference: en dan gaat hij dan gaat de dokter en de meisje bloemetjes zaaien bloemetjes xxx
Prediction: en dan gaat de meisje bloemd staan en dan toch ziet hij bloemde bloem

2
File Name: c20.mp3
Original Transcript: en de dokter die plukt een bloem 
Reference: en de dokter die plukt een bloem
Prediction: en de dochter die klopt op loon

3
File Name: c20.mp3
Original Transcript: en hij geeft de bloem aan dat meisje 
Reference: en hij geeft de bloem aan dat meisje
Prediction: en ik heb het blauw gevoel

4
File Name: c20.mp3
Original Transcript: en hij doet hem op zijn hoofd 
Reference: en hij doet hem op zijn hoofd
Prediction: kan hij dit van ons hebben hoopt heel goed nou we hebben een behoud klaar dat 

In [19]:
del_idx = []
for i in range(len(result)):
  reference = result[i]['reference'].split()
  prediction = result[i]['prediction'].split()

  if len(prediction) > 2 * len(reference):
    print(reference)
    print(prediction)
    print('')
    del_idx.append(i)

result = result.select(
    (
        i for i in range(len(result))
        if i not in set(del_idx)
    )
)

['en', 'hij', 'doet', 'hem', 'op', 'zijn', 'hoofd']
['kan', 'hij', 'dit', 'van', 'ons', 'hebben', 'hoopt', 'heel', 'goed', 'nou', 'we', 'hebben', 'een', 'behoud', 'klaar', 'dat', 'ging', 'heel', 'goed', 'mag', 'jij', 'even', 'mijn', 'stukje', 'uitzoeken', 'welke', 'kleur', 'wordt', 'het', 'en', 'blauw', 'zal', 'ik', 'het', 'even', 'afhalen', 'waar', 'wil', 'je', 'hem', 'op', 'je', 'blaadje', 'hebben', 'in', 'de', 'hoek', 'zo', 'in', 'de', 'pak']

['die', 'heeft', 'de', 'bal', 'in', 'het', 'net']
['je', 'hebt', 'de', 'bal', 'in', 'het', 'bed', 'en', 'je', 'had', 'die', 'bij', 'je', 'laatste', 'bal', 'de', 'spits', 'toch', 'ja', 'hij', 'is', 'weer', 'blij', 'en', 'kan', 'weer', 'voetballen', 'nou', 'dat', 'was', 'er', 'alweer', 'een', 'er', 'komt', 'er', 'weer', 'een', 'stukje', 'bij', 'en', 'groene', 'die', 'zie', 'je', 'nog', 'zie', 'je', 'dat']

['en', 'daar', 'heeft', 'de', 'piraat', 'de', 'bal', 'weer', 'terug']
['en', 'groene', 'die', 'zie', 'je', 'nog', 'zie', 'je', 'dat', 'waar',

Parameter 'indices'=<generator object <genexpr> at 0x7cb56c26b680> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [20]:
len(result)

327

In [21]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.1 MB/s eta 0:00:00


In [22]:
wer = load("wer")
print(wer.compute(references=result["reference"], predictions=result["prediction"]))

0.6347962382445141
